# SCE-TTS: Glow-TTS 학습

이 문서는 SCE-TTS 프로젝트의 Glow-TTS 학습용 문서입니다.

이 데모에 대한 더 자세한 정보는 아래 링크에서 확인하실 수 있습니다.  
https://sce-tts.github.io/

실제 학습을 진행하시려면, 위쪽 메뉴에서 `런타임 -> 모두 실행`을 클릭하신 후,  
문서 최하단의 `Glow-TTS 학습 진행`까지 정상적으로 실행되는걸 확인해주시면 됩니다.

도중에 문제가 발생한다면, 문제의 원인을 해결한 후 `런타임 -> 런타임 초기화`를 클릭하고,  
다시 `런타임 -> 모두 실행`을 클릭해주세요.

In [ ]:
출처: https://sce-tts.github.io/#/v2/train

## 1. 할당된 GPU 확인

현재 런타임에 할당된 GPU를 확인합니다.

만약, `GPU: NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.` 라는 메시지가 출력된다면, 위쪽 메뉴에서 `런타임 -> 런타임 유형 변경`을 클릭하고 하드웨어 가속기를 `GPU`로 변경하여 저장한 후 다시 실행해주세요.

In [ ]:
import os
GPU_NAME = os.popen('nvidia-smi --query-gpu=name --format=csv,noheader').read().strip()
os.environ['GPU_NAME'] = GPU_NAME
print(f'GPU: {GPU_NAME}')

GPU: Tesla P100-PCIE-16GB


## 2. 구글 드라이브 마운트

음성합성을 위해 학습한 모델이 있는 구글 드라이브를 마운트합니다.  
마운트할 구글 드라이브 내에 다음 파일이 존재하는지 꼭 확인해주세요.

- `/Colab Notebooks/data/filelists.zip`

(존재하지 않는다면, [음성 데이터셋 변환](https://sce-tts.github.io/#/recoding?id=%EC%9D%8C%EC%84%B1-%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B-%EB%B3%80%ED%99%98)문서를 참조하여 준비해주세요.)

만약 아래에 `Enter your authorization code:`과 같은 메시지가 출력될 경우,  
같이 출력된 링크에 접속하여, 마운트할 구글 계정을 선택하신 후, 인증 코드를 복사하여 입력해주세요.

In [ ]:
  from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 3. 필수 라이브러리 및 함수 불러오기

실행에 필요한 라이브러리 및 함수를 불러옵니다.

이 과정은 약 10분 정도 소요될 수 있습니다.

In [ ]:
import sys
from pathlib import Path

In [ ]:
%cd /content
!git clone --depth 1 https://github.com/sce-tts/TTS.git -b sce-tts
%cd /content/TTS
!pip install -q --no-cache-dir -e .

/content
Cloning into 'TTS'...
remote: Enumerating objects: 447, done.
remote: Counting objects: 100% (447/447), done.
remote: Compressing objects: 100% (416/416), done.
remote: Total 447 (delta 55), reused 247 (delta 19), pack-reused 0
Receiving objects: 100% (447/447), 13.77 MiB | 32.19 MiB/s, done.
Resolving deltas: 100% (55/55), done.
/content/TTS
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 183 kB 7.7 MB/s 
     |████████████████████████████████| 228 kB 89.7 MB/s 
     |████████████████████████████████| 69 kB 103.7 MB/s 
     |████████████████████████████████| 124 kB 101.2 MB/s 
     |████████████████████████████████| 1.3 MB 67.8 MB/s 
     |████████████████████████████████| 3.2 MB 65.9 MB/s 
     |████████████████████████████████| 71 kB 78.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend depend

## 4. 학습할 데이터셋 불러오기

학습에 사용할 음성 데이터를 구글 드라이브에서 가져옵니다.

In [ ]:
%cd /content/TTS
!cp "/content/drive/My Drive/Colab Notebooks/data/filelists.zip" ./filelists.zip
!rm -rf ./filelists
!unzip -q filelists.zip -d ./filelists

/content/TTS


## 5. 사전 학습 데이터 불러오기

사전 학습 데이터가 구글 드라이브에 존재하지 않을 경우,  
SleepingCE Speech Dataset의 사전 학습 데이터를 내려받습니다.

In [ ]:
%cd /content/TTS
!mkdir -p "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2"
if not Path("/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/config.json").exists():
    !gdown --id 1DMKLdfZ_gzc_z0qDod6_G8fEXj0zCHvC -O glowtts-v2.zip
    !unzip -q glowtts-v2.zip -d ./
    !cp -R ./glowtts-v2/* "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/"

/content/TTS


In [ ]:
%cd /content/TTS
if not Path("/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/scale_stats_new.npy").exists():
    !python TTS/bin/compute_statistics.py "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/config.json" "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/scale_stats_new.npy" --data_path "/content/TTS/filelists/wavs/"

/content/TTS


In [ ]:
with open("/content/TTS/test_sentences.txt", mode="w") as f:
    f.write("""아래 문장들은 모델 학습을 위해 사용하지 않은 문장들입니다.
서울특별시 특허허가과 허가과장 허과장.
경찰청 철창살은 외철창살이고 검찰청 철창살은 쌍철창살이다.
지향을 지양으로 오기하는 일을 지양하는 언어 습관을 지향해야 한다.
그러니까 외계인이 우리 생각을 읽고 우리 생각을 우리가 다시 생각토록 해서 그 생각이 마치 우리가 생각한 것인 것처럼 속였다는 거냐?""")

## 6. TensorBoard 실행

학습 진행을 확인하기 위해 TensorBoard를 실행합니다.

최신 학습 진행 상황을 확인하려면 TensorBoard 우측 상단의 새로고침 아이콘을 클릭해주세요.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir="/content/drive/My Drive/Colab Notebooks/data/glowtts-v2"

<IPython.core.display.Javascript object>

## 7. Glow-TTS 학습 진행

실제 Glow-TTS 학습을 진행합니다.

학습이 정상적으로 진행되면, 이 셀은 종료되지 않고 계속 실행되는 상태를 유지합니다.

학습 진행 정도를 시각적으로 확인하시려면 바로 위의 TensorBoard를 참고해주세요.

학습된 결과를 확인하고 싶으시다면, 가장 최근 체크포인트를 [SCE-TTS: 음성합성 데모](https://colab.research.google.com/drive/1YkxjzBz3V4eXoAaEgcFNEUg8ZyWV40x9)에서 불러와 확인할 수 있습니다.

처음 학습을 수행할 때에는 아래 셀을 그대로 실행하면 됩니다.

이전에 학습을 진행하던 모델을 이어서 학습을 진행하시려면 다음과 같이 수정한 후 실행해주세요.

- 아래 셀에서 2 ~ 3번째 줄의 코드를 주석을 해제합니다.
- 3번째 줄의 경로를 이어서 학습을 진행할 모델의 경로로 변경합니다.  
(예시: `/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/glowtts-v2-May-31-2021_08+17AM-d897f2e`)
- 4번째 줄 아래의 코드를 제거합니다.


In [ ]:
%cd /content/TTS
!(python TTS/bin/train_glow_tts.py \
    --continue_path "/content/drive/MyDrive/Colab Notebooks/data/glowtts-v2/glowtts-v2-August-10-2021_05+52AM-3aa165a")
# !(python TTS/bin/train_glow_tts.py \
#     --config_path "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/config.json" \
#     --coqpit.datasets.0.path "/content/TTS/filelists"  \
#     --coqpit.audio.stats_path "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/scale_stats_new.npy"  \
#     --coqpit.test_sentences_file "/content/TTS/test_sentences.txt"  \
#     --coqpit.output_path "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/"  \
#     --coqpit.num_loader_workers 2  \
#     --coqpit.num_val_loader_workers 2  \
#     --restore_path "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/model_file.pth.tar")